### Librairies

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

pd.options.display.max_columns = 999

### Lecture des données

In [2]:
df_labels = pd.read_json ('deezer_data/album_genres_new_releases.json' , lines = True )
df_data = pd.read_json ('deezer_data/sampled_geoloc_counters_new_releases.json' , lines = True )

### Pré-traitement des données

#### 1) Table Labels

In [4]:
binarized_df_labels = pd.get_dummies( df_labels , columns=['genre_name'] , prefix="" , prefix_sep="" )
df_multi_labels = binarized_df_labels.groupby(['album_id'] , as_index=False ).sum()
df_multi_labels.head()

,album_id,Afro Pop,Alternative,Blues,Chanson française,Chill Out/Trip-Hop/Lounge,Classique,Comédies musicales,Country,Dance,Dancehall/Ragga,Disco,Dub,East Coast,Electro,Electro Pop/Electro Rock,Films/Jeux vidéo,Folk,Gospel,Hard Rock,Jazz,Jeunesse,Latino,Metal,Musique africaine,Musique arabe,Musique brésilienne,Musique religieuse,Musiques de films,Música colombiana,Opéra,Pop,Pop Indé,Pop indé/Folk,Pop internationale,Pop latine,R&B,Rap français,Rap/Hip Hop,Reggae,Rock,Rock & Roll/Rockabilly,Rock français,Rock indé,Singer & Songwriter,Soul,Soul & Funk,Sports,Techno/House,Trance,Variété Internationale,World
0,13972552,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,13979628,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,13986414,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,13994594,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,13994632,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### 2) Table des données spatio-temporelle

In [5]:
df_data_ignoreTime = df_data.groupby(['album_id','loc_city'] , as_index=False )['nstreams','nusers'].sum()
df_data_ignoreTime_binarized =  pd.get_dummies( df_data_ignoreTime ,
                                               columns=['loc_city'] ,
                                               prefix="" , prefix_sep="" ).groupby(['album_id'] ,
                                                                                   as_index=False ).sum()
df_data_ignoreTime_binarized.head()

,album_id,nstreams,nusers,A Dos Cunhados,Abbeville,Abidjan,Ablon-sur-seine,Abrest,Abymes,Acheres,Achicourt,Achères,Achères-la-forêt,Adissan,Afa,Agadir,Agde,Agen,Agnetz,Ahmedabad,Aiffres,Aire-sur-la-lys,Aix-en-Provence,Aix-en-provence,Aix-les-Bains,Aix-les-bains,Aix-noulette,Aixe-sur-vienne,Aizenay,Ajaccio,Alata,Albertville,Albi,Albon,Alençon,Alfortville,Algiers,Algrange,Alicante,Alixan,Allauch,Allennes-les-Marais,Allennes-les-marais,Allevard,Allonnes,Almaty,Altkirch,Alès,Amarens,Ambilly,Amboise,Ambérieu-en-bugey,Ametlla De Mar,Amiens,Amilly,Amnéville,Ampuis,Amsterdam,Ancenis,Andard,Andorra La Vella,Andresy,Andrezieux-Boutheon,Andrésy,Andrézieux,Anduze,Angers,Angerville,Angles,Anglet,Angoulême,Angres,Aniane,Aniche,Ankara,Annecy,Annecy-le-Vieux,Annecy-le-vieux,Annemasse,Annequin,Annoeullin,Annonay,Anse,Antalya,Anthon,Antibes,Antony,Antwerpen,Anzin,Aoste,Appietto,Apprieu,Aprelevka,Apt,Arbusigny,Arcachon,Archamps,Arcis-sur-aube,Arcueil,Ardon,Arezzo,Argelès-sur-mer,Argences,Argens,Argenteuil,Argonay,Arles,Arleux,Armavir,Armbouts-cappel,Armentières,Arnouville,Arpajon,Arras,Ars-sur-moselle,Artigues-pres-Bordeaux,Artigues-près-bordeaux,Asfeld,Ashburn,Aslonnes,Asnieres-sur-Seine,Asnières,Aspiran,Aspremont,Astoria,Athens,Athis-Mons,Athis-de-l'orne,Athis-mons,Attiches,Aubagne,Aubenas,Aubencheul-aux-bois,Aubergenville,Aubers,Aubervilliers,Aubevoye,Aubignan,Aubière,Auby,Aubérives-sur-varèze,Aucamville,Auch,Auchel,Auchy-les-mines,Auckland,Audenge,Audincourt,Audun-le-tiche,Aulnay-sous-Bois,Aulnay-sous-bois,Aulnois-sous-laon,Aulnoye-aymeries,Auray,Aurillac,Auriol,Aussonne,Austin,Auterive,Authume,Autry-le-châtel,Autun,Auvers-saint-georges,Auvers-sur-oise,Auxerre,Auxonne,Avallon,Avermes,Avignon,Avion,Avize,Avranches,Avrillé,Babson Park,Bagneux,Bagnolet,Bagnols-sur-Ceze,Bagnols-sur-cèze,Baguio City,Baie-Mahault,Baie-mahault,Baillargues,Bailleul,Bailleval,Bailly,Bailly-romainvilliers,Baku,Balaruc-les-Bains,Balasore,Ballan-miré,Ballancourt,Ballancourt-sur-Essonne,Balma,Ban-saint-martin,Bandol,Bandung,Bangalore,Bangkok,Bar-le-Duc,Bar-le-duc,Barakaldo,Barberaz,Barcelona,Barentin,Bari,Barjouville,Barking,Barlin,Baron,Barr,Basauri,Basingstoke,Basse-Goulaine,Basse-goulaine,Basse-ham,Bastelicaccia,Bastia,Baud,Baule,Baume-les-dames,Bauvin,Bavans,Bayard-sur-marne,Bayeux,Bayonne,Beaugency,Beaumont,Beaumont-sur-Oise,Beaumont-sur-oise,Beaune,Beaupréau,Beaurepaire,Beausemblant,Beausoleil,Beauvais,Beauvoir-sur-niort,Beauzelle,Beaver,Bechyne,Begles,Beijing,Beirut,Belfort,Belgorod,Belgrade,Bellegarde-sur-Valserine,Bellegarde-sur-valserine,Bellerive-sur-allier,Belleville-sur-saône,Belley,Benfeld,Benglen,Berck,Bergamo,Bergen,Bergerac,Berlin,Bernardswiller,Bernay,Berre-l'Etang,Berre-l'etang,Besançon,Besnans,Bessan,Betton,Bezons,Biarritz,Bidart,Bienville,Biganos,Bihorel,Bilbao,Billancourt,Billere,Billom,Billy,Billère,Binic,Biot,Birmingham,Bischheim,Bischoffsheim,Bischwihr,Bischwiller,Biviers,Blagnac,Blain,Blaise,Blida,Blois,Blénod-lès-pont-à-mousson,Blénod-lès-toul,Bobigny,Bogotá,Boigny,Bois,Bois-Colombes,Bois-Guillaume,Bois-bernard,Bois-colombes,Bois-guillaume,Boissy-Saint-Léger,Boissy-saint-léger,Bolbec,Bollate,Bollene,Bollène,Bon-encontre,Bondoufle,Bondy,Bonneuil-en-france,Bonneuil-sur-marne,Bonneville,Bons,Bons-en-Chablais,Bonsecours,Bordeaux,Borehamwood,Borgo,Bosc-le-hard,Bossieu,Bouc-Bel-Air,Bouc-bel-air,Bouchemaine,Boufféré,Bougival,Bouguenais,Boulay,Boulbon,Bouliac,Bouloc,Boulogne,Boulogne-Billancourt,Boulogne-billancourt,Boulogne-sur-Mer,Boulogne-sur-mer,Bourbriac,Bourg-en-Bresse,Bourg-en-bresse,Bourg-la-Reine,Bourg-la-reine,Bourg-les-Valence,Bourg-lès-valence,Bourges,Bourgoin,Bourguébus,Bousselange,Boussy-saint-antoine,Bouxières-aux-chênes,Bouxières-aux-dames,Bouzonville,Boves,Boëge,Braintree,Bram,Bratsk,Brazey-en-plaine,Bressols,Brest,Breteuil,Breuil-le-sec,Brie-Comte-Robert,Brie-comte-robert,Briey,Brignais,Brignoles,Brin-sur-seille,Brindas,Brioude,Brisbane,Brissac,Brive-la-Gaillarde,Brive-la-gaillarde,Brockport,Bron,Brooklyn,Brue-auriac,Bruguieres,Brugui

#### 3) Jointure

In [6]:
final_df = pd.merge(df_data_ignoreTime_binarized , df_multi_labels , on='album_id')
print(final_df.shape)
final_df.head()

(1543, 2737)


,album_id,nstreams,nusers,A Dos Cunhados,Abbeville,Abidjan,Ablon-sur-seine,Abrest,Abymes,Acheres,Achicourt,Achères,Achères-la-forêt,Adissan,Afa,Agadir,Agde,Agen,Agnetz,Ahmedabad,Aiffres,Aire-sur-la-lys,Aix-en-Provence,Aix-en-provence,Aix-les-Bains,Aix-les-bains,Aix-noulette,Aixe-sur-vienne,Aizenay,Ajaccio,Alata,Albertville,Albi,Albon,Alençon,Alfortville,Algiers,Algrange,Alicante,Alixan,Allauch,Allennes-les-Marais,Allennes-les-marais,Allevard,Allonnes,Almaty,Altkirch,Alès,Amarens,Ambilly,Amboise,Ambérieu-en-bugey,Ametlla De Mar,Amiens,Amilly,Amnéville,Ampuis,Amsterdam,Ancenis,Andard,Andorra La Vella,Andresy,Andrezieux-Boutheon,Andrésy,Andrézieux,Anduze,Angers,Angerville,Angles,Anglet,Angoulême,Angres,Aniane,Aniche,Ankara,Annecy,Annecy-le-Vieux,Annecy-le-vieux,Annemasse,Annequin,Annoeullin,Annonay,Anse,Antalya,Anthon,Antibes,Antony,Antwerpen,Anzin,Aoste,Appietto,Apprieu,Aprelevka,Apt,Arbusigny,Arcachon,Archamps,Arcis-sur-aube,Arcueil,Ardon,Arezzo,Argelès-sur-mer,Argences,Argens,Argenteuil,Argonay,Arles,Arleux,Armavir,Armbouts-cappel,Armentières,Arnouville,Arpajon,Arras,Ars-sur-moselle,Artigues-pres-Bordeaux,Artigues-près-bordeaux,Asfeld,Ashburn,Aslonnes,Asnieres-sur-Seine,Asnières,Aspiran,Aspremont,Astoria,Athens,Athis-Mons,Athis-de-l'orne,Athis-mons,Attiches,Aubagne,Aubenas,Aubencheul-aux-bois,Aubergenville,Aubers,Aubervilliers,Aubevoye,Aubignan,Aubière,Auby,Aubérives-sur-varèze,Aucamville,Auch,Auchel,Auchy-les-mines,Auckland,Audenge,Audincourt,Audun-le-tiche,Aulnay-sous-Bois,Aulnay-sous-bois,Aulnois-sous-laon,Aulnoye-aymeries,Auray,Aurillac,Auriol,Aussonne,Austin,Auterive,Authume,Autry-le-châtel,Autun,Auvers-saint-georges,Auvers-sur-oise,Auxerre,Auxonne,Avallon,Avermes,Avignon,Avion,Avize,Avranches,Avrillé,Babson Park,Bagneux,Bagnolet,Bagnols-sur-Ceze,Bagnols-sur-cèze,Baguio City,Baie-Mahault,Baie-mahault,Baillargues,Bailleul,Bailleval,Bailly,Bailly-romainvilliers,Baku,Balaruc-les-Bains,Balasore,Ballan-miré,Ballancourt,Ballancourt-sur-Essonne,Balma,Ban-saint-martin,Bandol,Bandung,Bangalore,Bangkok,Bar-le-Duc,Bar-le-duc,Barakaldo,Barberaz,Barcelona,Barentin,Bari,Barjouville,Barking,Barlin,Baron,Barr,Basauri,Basingstoke,Basse-Goulaine,Basse-goulaine,Basse-ham,Bastelicaccia,Bastia,Baud,Baule,Baume-les-dames,Bauvin,Bavans,Bayard-sur-marne,Bayeux,Bayonne,Beaugency,Beaumont,Beaumont-sur-Oise,Beaumont-sur-oise,Beaune,Beaupréau,Beaurepaire,Beausemblant,Beausoleil,Beauvais,Beauvoir-sur-niort,Beauzelle,Beaver,Bechyne,Begles,Beijing,Beirut,Belfort,Belgorod,Belgrade,Bellegarde-sur-Valserine,Bellegarde-sur-valserine,Bellerive-sur-allier,Belleville-sur-saône,Belley,Benfeld,Benglen,Berck,Bergamo,Bergen,Bergerac,Berlin,Bernardswiller,Bernay,Berre-l'Etang,Berre-l'etang,Besançon,Besnans,Bessan,Betton,Bezons,Biarritz,Bidart,Bienville,Biganos,Bihorel,Bilbao,Billancourt,Billere,Billom,Billy,Billère,Binic,Biot,Birmingham,Bischheim,Bischoffsheim,Bischwihr,Bischwiller,Biviers,Blagnac,Blain,Blaise,Blida,Blois,Blénod-lès-pont-à-mousson,Blénod-lès-toul,Bobigny,Bogotá,Boigny,Bois,Bois-Colombes,Bois-Guillaume,Bois-bernard,Bois-colombes,Bois-guillaume,Boissy-Saint-Léger,Boissy-saint-léger,Bolbec,Bollate,Bollene,Bollène,Bon-encontre,Bondoufle,Bondy,Bonneuil-en-france,Bonneuil-sur-marne,Bonneville,Bons,Bons-en-Chablais,Bonsecours,Bordeaux,Borehamwood,Borgo,Bosc-le-hard,Bossieu,Bouc-Bel-Air,Bouc-bel-air,Bouchemaine,Boufféré,Bougival,Bouguenais,Boulay,Boulbon,Bouliac,Bouloc,Boulogne,Boulogne-Billancourt,Boulogne-billancourt,Boulogne-sur-Mer,Boulogne-sur-mer,Bourbriac,Bourg-en-Bresse,Bourg-en-bresse,Bourg-la-Reine,Bourg-la-reine,Bourg-les-Valence,Bourg-lès-valence,Bourges,Bourgoin,Bourguébus,Bousselange,Boussy-saint-antoine,Bouxières-aux-chênes,Bouxières-aux-dames,Bouzonville,Boves,Boëge,Braintree,Bram,Bratsk,Brazey-en-plaine,Bressols,Brest,Breteuil,Breuil-le-sec,Brie-Comte-Robert,Brie-comte-robert,Briey,Brignais,Brignoles,Brin-sur-seille,Brindas,Brioude,Brisbane,Brissac,Brive-la-Gaillarde,Brive-la-gaillarde,Brockport,Bron,Brooklyn,Brue-auriac,Bruguieres,Brugui

In [7]:
from sklearn import tree
from sklearn.model_selection import train_test_split

# supprimer l'album_id qui nous intèresse pas
data = final_df.values[:,1:]

# le -1 c'est a cause de l'album_id 
features = data [ : , : df_data_ignoreTime_binarized.shape[1] - 1 ]
labels  = data [ :  , df_data_ignoreTime_binarized.shape[1] - 1  :  ]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    features,
    labels,
    test_size=0.2,
    random_state=1234,
)

In [9]:
# random_state  : pour fixer les mêmes résultats randoms 
clf  = tree.DecisionTreeClassifier( criterion='entropy',random_state=1234 )
clf.fit(X=X_train, y=y_train)
print ("accuracy of a decision Tree ",clf.score(X=X_test, y=y_test) ) 

accuracy of a decision Tree  0.22006472491909385
